In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from bin import *

from IPython.display import clear_output
%load_ext autoreload
%autoreload 2

In [2]:
mpl.use("pgf")

mpl.rcParams.update({
    "pgf.texsystem": "lualatex",
    'font.family': 'serif',
    'font.size' : 8,
    'text.usetex': True,
    'pgf.rcfonts': False,
    'pgf.preamble': tex_preamble,
})

In [3]:
n = 10000
expr1 = {
    'name' : 'runge_uniform_spec',
    'lam': np.linspace(-1,1,n,endpoint=False)+1/n,
    'k_max': 50,
    'k_skip': 1,
    'd': 500,
    'f': lambda x: 1/(1+16*x**2)
}

n = n//4
η=.25
expr2 = {
    'name' : 'runge_gap_spec',
    'lam': np.hstack([np.linspace(-1,-1+η,n//2,endpoint=False)+η/n,\
                      np.linspace(1-η,1,n//2,endpoint=False)+η/n])
,
    'k_max': 50,
    'k_skip': 1,
    'd': 500,
    'f': lambda x: 1/(1+16*x**2)
}


expr3 = {
    'name' : 'model_problem',
    'lam': model_problem_spectrum(300,.7,1e3),
    'k_max': 60,
    'k_skip': 1,
    'd':1000,
    'f': lambda x: 1/x
}

expr4 = {
    'name' : 'MNIST',
    'lam': np.load('matrices/MNIST_cov_eigs.npy').flatten(),
    'k_max': 65,
    'k_skip': 1,
    'd': 6000,
    'f': lambda x: (x>49907)
}


exprs = [expr1,expr2,expr3,expr4]

In [4]:
GQ_err = []
GQro_err = []
IQ_err = []
HQ_err = []
AQ_err = []

for expr_idx,expr in enumerate(exprs):
    
    lam = expr['lam']
    k_max = expr['k_max']
    k_skip = expr['k_skip']
    d = expr['d']
    n = len(lam)
    a,b = np.min(lam),np.max(lam)
    
    A = sp.sparse.spdiags(lam,0,n,n)

    v = np.ones(n)
    v/=np.linalg.norm(v)

    f = expr['f']

    
    GQ_err.append(np.full(k_max,np.nan))
    GQro_err.append(np.full(k_max,np.nan))
    IQ_err.append(np.full(k_max,np.nan))
    HQ_err.append(np.full(k_max,np.nan))
    AQ_err.append(np.full(k_max,np.nan))
    
    Q,(α,β) = lanczos_reorth(A,v,k_max)
    Q,(αro,βro) = lanczos_reorth(A,v,min(k_max,n),reorth=k_max)

    print(np.max(np.abs(α[:min(k_max,n)]-αro)),np.max(np.abs(αro)),\
          np.max(np.abs(β[:min(k_max,n)]-βro)),np.max(np.abs(βro)))
    
    αT = np.ones(max(2*k_max,d))*(a+b)/2
    βT = np.ones(max(2*k_max,d))*(b-a)/4
    βT[0] *= np.sqrt(2)
    m = get_chebyshev_moments(A,v,k_max,a,b)
    m_lan = get_moments_from_lanczos(A,v,2*k_max,αT,βT,T=(α,β))
    
    print(np.max(np.abs(m-m_lan)))

    GQ_err[-1][0] = v.T@(f(lam)*v)
    GQro_err[-1][0] = v.T@(f(lam)*v)
    IQ_err[-1][0] = v.T@(f(lam)*v)
    AQ_err[-1][0] = v.T@(f(lam)*v)
    HQ_err[-1][0] = v.T@(f(lam)*v)
    for k in range(k_skip,k_max,k_skip):
        
        θ,ω = get_gq(k,α,β)
        GQ_err[-1][k] = ω@f(θ) - v.T@(f(lam)*v)
        
        θ,ω = get_gq(k,αro,βro)
        GQro_err[-1][k] = ω@f(θ) - v.T@(f(lam)*v)

        s = 2*k
        θ,ω = get_iq(m,s,αT,βT)
        IQ_err[-1][k] = ω@f(θ) - v.T@(f(lam)*v)
    
        θ,ω = get_aaq(m,s,d,αT,βT)
        AQ_err[-1][k] = ω@f(θ) - v.T@(f(lam)*v)

        αH = np.ones(2*k_max)*(a+b)/2
        βH = np.ones(2*k_max)*(b-a)/4
        αH[:k] = α[:k]
        βH[:k] = β[:k]
        
        mH = get_moments_from_lanczos(A,v,2*k,αH,βH,T=(α,β))
        θ,ω = get_iq(mH,s,αH,βH)
        HQ_err[-1][k] = ω@f(θ) - v.T@(f(lam)*v)

2.220446049250313e-16 3.608224830031759e-16 1.1102230246251565e-16 0.5773502663028742
1.7277845820728999e-15
1.5460722979643293e-15 9.20053963571199e-16 2.220446049250313e-16 0.8779711441727451
5.9396931817445875e-15
934.2147017079998 769.9811669986356 467.16162186883565 262.85091181293865
2.2093438190040615e-13
249861.85127229986 210493.06774408638 146147.7860976836 97438.0727897988
3.4861002973229915e-14


In [5]:
for expr_idx,expr in enumerate(exprs):
    
    fig = plt.figure(figsize=(figure_width*cm,figure_height*cm))

    ax = fig.add_axes([left, bottom, width, height])

    lam = expr['lam']
    k_max = expr['k_max']
    n = len(lam)
    a,b = min(lam),max(lam)
    
    ax.plot(np.abs(GQro_err[expr_idx]),\
            **line_styles['l2'],label='GQ reorth')
    ax.plot(np.abs(GQ_err[expr_idx]),\
            **line_styles['l1'],label='GQ')
    ax.plot(np.abs(IQ_err[expr_idx]),\
            **line_styles['l4'],label='IQ')
    ax.plot(np.abs(AQ_err[expr_idx]),\
            **line_styles['l3'],label='AQ')
#    ax.plot(np.abs(AQ_err[expr_idx]),**line_styles['l3'],ls='None')
#    ax.plot(2*np.abs(err_ℐ[expr_idx][::2]),**line_styles['l1'],ls='None')
    
#    ax.legend()
    ax.grid(True,linestyle=':',linewidth=.5)

    ax.set_xlabel('number of matvec: $k$')
    ax.set_ylabel(r'error: $\textstyle\big|\int f \,\mathrm{d}\Psi-\int f \,\mathrm{d}[\Psi]_k^\circ\,\big|$')

    ax.set_yscale('log')
    
    plt.savefig(f'imgs/ch3_{expr["name"]}.pdf')
    plt.close()

In [6]:
err_ℐ = []
for expr_idx,expr in enumerate(exprs):
    print(expr_idx)
    lam = expr['lam']
    k_max = expr['k_max']
    k_skip = expr['k_skip']
    d = expr['d']
    f = expr['f']

    n = len(lam)
    a,b = np.min(lam),np.max(lam)
    
    err_ℐ.append(np.full(2*k_max,np.nan))
    ℐ = np.linspace(a,b,10000)

    i=[1]
    for k in range(2*k_max):
        i_add = np.argmax(np.min(np.abs(ℐ[i]-ℐ[:,None]),axis=1))
        i = np.append(i,i_add)
        i = np.sort(i)
        i,p,ub,lb = remez(f,ℐ,k,i=i,tol=1e-10,max_iter=20)
        err_ℐ[-1][k] = ub

0
1
2
3


/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified tollerence for degree 3
  warnings.warn(f"failed to converge to specified tollerence for degree {k}")
/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified tollerence for degree 4
  warnings.warn(f"failed to converge to specified tollerence for degree {k}")
/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified tollerence for degree 5
  warnings.warn(f"failed to converge to specified tollerence for degree {k}")
/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified tollerence for degree 6
  warnings.warn(f"failed to converge to specified tollerence for degree {k}")
/home/tyler/Documents/GitHub/Research_Code/thesis/code/bin/remez.py:37: UserWarning: failed to converge to specified tollere